<a href="https://colab.research.google.com/github/isegura/OCW-UC3M-NLPDeep-2023/blob/main/tema6_3_backtranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>   

# 6.3. Back translation para generar textos sintéticos

En este ejercicio, vamos a estudiar una técnica de generación automática de textos basada en "back translation".
La idea básica de este método es traducir el texto de entrada a un idioma, y volver a traducirlo al idioma original. De esta forma, podemos generar un nuevo texto con diferentes palabras conservando el significado del texto original. Por ejemplo:

- **Texto original**: *No me esperes para el almuerzo* => (español a inglés) *Don’t wait me for lunch* => (de inglés a español)
- **Texto generado**:  *No me esperes para almorzar*.

En el ejemplo anterior, podemos ver que el texto generado tiene el mismo significado que el texto original, y además no tiene errores sintácticos.









## Back-translation con textaugment

Algunas librerías de Python ya implementan esta técnica. Por ejemplo, la librería **textaugment**, proporciona una clase **Translate** para generar un nuevo texto usando back translation. Es necesario indicar el idioma de origen, y el idioma que se utiliza como paso intermedio.


In [1]:
! pip install -q textaugment

In [2]:
from textaugment import Translate
SRC = "es" # idioma de los textos
TO = "en" # idioma que se utiliza para traducir como paso intermedio


texts = ["El cambio climático unido al fenómeno de ‘El Niño’ impulsan las temperaturas globales hasta marcas nunca documentadas.",
         "Cada vez son más tristes las canciones de amor.",
         "Pozuelo de Alarcón, en Madrid, repite como la localidad con la renta más alta, de 80.244 euros, frente a los 12.737 de Higuera de Vargas (Badajoz).",
         "La Cumbre de Granada cobra extrema relevancia porque formalmente marcará el comienzo del debate al más alto nivel sobre la capacidad de los Veintisiete para integrar nuevos miembros.",
         'El bono estadounidense llega a máximos desde 2007 y el alemán, a su mayor cota desde 2011.',
         'El presidente de Ucrania realiza su primera visita a España desde que comenzó la guerra.'
    ]

t = Translate(src=SRC, to=TO)
for text in texts:
    print('Texto original:', text)
    print('Texto generado:', t.augment(text))
    print()


Texto original: El cambio climático unido al fenómeno de ‘El Niño’ impulsan las temperaturas globales hasta marcas nunca documentadas.
Texto generado: el cambio climático vinculado al fenómeno de "el niño" promueve las temperaturas globales a las marcas nunca documentadas.

Texto original: Cada vez son más tristes las canciones de amor.
Texto generado: las canciones de amor son cada vez más tristes.

Texto original: Pozuelo de Alarcón, en Madrid, repite como la localidad con la renta más alta, de 80.244 euros, frente a los 12.737 de Higuera de Vargas (Badajoz).
Texto generado: pozuelo de alarcón, en madrid, repite cómo la ciudad con el mayor ingreso, de 80,244 euros, en comparación con los 12,737 de higuera de vargas (badajoz).

Texto original: La Cumbre de Granada cobra extrema relevancia porque formalmente marcará el comienzo del debate al más alto nivel sobre la capacidad de los Veintisiete para integrar nuevos miembros.
Texto generado: la cumbre de granada cobra relevancia extrema 

Podemos ver que las oraciones generadas mantienen el significado de las oraciones originales. Además, en general, son correctas sintácticamente.


## Implementando nuestro método de back-translation (usando googletrans)
También es posible implementar nosotros mismo un método que permita generar un nuevo texto usando este enfoque de back translation. Para ello, usaremos la librería de Python **googletrans**, que actúa como interfaz para llamar al API de Google translator.

Para que dicha librería funcione correctamente en Google Colab, es necesario instalar la versión 3.1.0a0, tal y como se indica en la siguiente celda:


In [3]:
#!pip3 uninstall googletrans
!pip3 install -q googletrans==3.1.0a0

Una vez instalada, vamos a ver cómo utilizar esta librería.

En principio, no sería necesario indicar el idioma de origen, porque la librería es capaz de detectar el idioma de un texto.
Puede tardar unos minutos.


In [4]:
from googletrans import Translator
#creamos el objeto translator

translator = Translator()
translation = translator.translate("Cada vez son más tristes las canciones de amor.", dest="en", src='es')
print(translation.origin, ' -> ', translation.text)


Cada vez son más tristes las canciones de amor.  ->  Love songs are getting sadder and sadder.


Ahora vamos a volver a traducir de inglés a español, usando el mismo método, y simplemente cambiando los argumentos de entrada:

In [5]:

translation = translator.translate(translation.text, dest="es", src='en')
print(translation.origin, ' -> ', translation.text)


Love songs are getting sadder and sadder.  ->  Las canciones de amor son cada vez más tristes.


Vamos a definir una función que reciba un texto, y devuelva el texto generado basado en este enfoque de back translation:

In [6]:
def back_translate(input_text, src_lang, to_lang) -> str:
    """gets an input text and translate it from src_lang to to_lang. Now, this translation
    is againg translated from to_lang to src_lang. This final translation is returned"""
    translator = Translator()
    translation = translator.translate(input_text, src=src_lang, dest=to_lang)
    # print(translation.origin, ' -> ', translation.text)
    translated_text = translation.text
    translation = translator.translate(translated_text, src=to_lang, dest=src_lang)
    return translation.text


Procesamos ahora nuestra colección de oraciones para generar  nuevos textos:

In [7]:
for input_text in texts:
    synthetic_text = back_translate(input_text,'es','en')
    print('Texto original:', input_text)
    print('Texto generado:', synthetic_text)
    print()

Texto original: El cambio climático unido al fenómeno de ‘El Niño’ impulsan las temperaturas globales hasta marcas nunca documentadas.
Texto generado: El cambio climático combinado con el fenómeno de 'El Niño' está elevando las temperaturas globales a niveles nunca antes documentados.

Texto original: Cada vez son más tristes las canciones de amor.
Texto generado: Las canciones de amor son cada vez más tristes.

Texto original: Pozuelo de Alarcón, en Madrid, repite como la localidad con la renta más alta, de 80.244 euros, frente a los 12.737 de Higuera de Vargas (Badajoz).
Texto generado: Pozuelo de Alarcón, en Madrid, repite como la localidad con mayores ingresos, 80.244 euros, frente a los 12.737 de Higuera de Vargas (Badajoz).

Texto original: La Cumbre de Granada cobra extrema relevancia porque formalmente marcará el comienzo del debate al más alto nivel sobre la capacidad de los Veintisiete para integrar nuevos miembros.
Texto generado: La Cumbre de Granada es sumamente relevante 

Te animo a que pruebes con otras otras APIs de traducción automática, como por ejemplo, Deepl (https://www.deepl.com/en/blog/announcing-python-client-library-for-deepl-api) o Yandex (https://cloud.yandex.com/en/docs/tracker/user/python).